# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN
...,...
498,XYL
499,YUM
500,ZBRA
501,ZBH


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrety import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f' https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
print(f'data status code is {data.status_code}')

data status code is 200


In [5]:
data = data.json()
print(f'{data}')

{'avgTotalVolume': 64666029, 'calculationPrice': 'close', 'change': 3.23, 'changePercent': 0.01765, 'close': 186.28, 'closeSource': 'official', 'closeTime': 1715630400016, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 186.305, 'delayedPriceTime': 1715630396032, 'extendedChange': -0.14, 'extendedChangePercent': -0.00075, 'extendedPrice': 186.14, 'extendedPriceTime': 1715644789740, 'high': 187.1, 'highSource': '15 minute delayed price', 'highTime': 1715630399998, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 186.285, 'iexCloseTime': 1715630399586, 'iexLastUpdated': 1715630399586, 'iexMarketPercent': 0.01837078921258574, 'iexOpen': 185.59, 'iexOpenTime': 1715607000699, 'iexRealtimePrice': 186.285, 'iexRealtimeSize': 68, 'iexVolume': 1323520, 'lastTradeTime': 1715630399638, 'latestPrice': 186.28, 'latestSource': 'Close', 'latestTime': 'May 13, 2024', 'latestUpdate': 1715630400016, 'latestVolume': 72044809, 'low': 184.62, 'lowSource': '15

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']
print(f'last price is {price}')
print(f'market cap is {market_cap}')

last price is 186.28
market cap is 2856432794960


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
new_data = {'Ticker': symbol, 
            'Stock Price': price, 
            'Market Capitalization': market_cap, 
            'Number of Shares to Buy': 'N/A'}

# Get the index for the new row
new_index = len(final_dataframe)

# Set the values for each column at the new index
for column, value in new_data.items():
    final_dataframe.at[new_index, column] = value
    
# check final_dataframe
final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,186.28,2856432794960,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [12]:
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'

    returnCode = requests.get(api_url)
    print(f'data status code is {returnCode.status_code} with stock {stock}')
    
    data = requests.get(api_url).json()
    price = data['latestPrice']
    market_cap = data['marketCap']

    new_data = {'Ticker': stock, 
                'Stock Price': price, 
                'Market Capitalization': market_cap, 
                'Number of Shares to Buy': 'N/A'}

    # Get the index for the new row
    new_index = len(final_dataframe)
    
    # Set the values for each column at the new index
    for column, value in new_data.items():
        final_dataframe.at[new_index, column] = value


data status code is 200 with stock MMM
data status code is 200 with stock AOS
data status code is 200 with stock ABT
data status code is 200 with stock ABBV
data status code is 200 with stock ACN
data status code is 200 with stock ADBE
data status code is 200 with stock AMD
data status code is 200 with stock AES
data status code is 200 with stock AFL
data status code is 200 with stock A
data status code is 200 with stock APD
data status code is 200 with stock ABNB
data status code is 200 with stock AKAM
data status code is 200 with stock ALB
data status code is 200 with stock ARE
data status code is 200 with stock ALGN
data status code is 200 with stock ALLE
data status code is 200 with stock LNT
data status code is 200 with stock ALL
data status code is 200 with stock GOOGL
data status code is 200 with stock GOOG
data status code is 200 with stock MO
data status code is 200 with stock AMZN
data status code is 200 with stock AMCR
data status code is 200 with stock AEE
data status code 

In [ ]:
final_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
def chunk(lst, n):
    """Yield sucessive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
symbol_groups = list(chunk(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        new_data = {'Ticker': symbol, 
            'Stock Price': data[symbol]['quote']['latestPrice'], 
            'Market Capitalization': data[symbol]['quote']['marketCap'], 
            'Number of Shares to Buy': 'N/A'}

        # Get the index for the new row
        new_index = len(final_dataframe)
        
        # Set the values for each column at the new index
        for column, value in new_data.items():
            final_dataframe.at[new_index, column] = value
final_dataframe

https://cloud.iexapis.com/stable/stock/market/batch?symbols=MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BK,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BX,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,BLDR,BG,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,COR,CNC,CNP,CF&types=quote&token=sk_29e5937776d842bfaf8c3fae352a10e7
https://cloud.iexapis.com/stable/stock/market/batch?symbols=CHRW,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CPAY,CTVA,CSGP,COST,CTRA,CCI,CSX,CMI,CVS,DHR,DRI,DVA,DAY,DECK,DE,DAL,DVN,DXCM,FANG,DLR,DFS,DG,DLTR,D,DPZ,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,LLY,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,EG,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FI

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,99.63,55131382035,N/A
1,AOS,86.06,13147861854,N/A
2,ABT,104.77,182261428930,N/A
3,ABBV,161.28,284799155720,N/A
4,ACN,307.41,206094347708,N/A
...,...,...,...,...
498,XYL,141.27,34250524844,N/A
499,YUM,136.38,38409001073,N/A
500,ZBRA,316,16248531348,N/A
501,ZBH,119.59,24603059117,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [14]:
portfolio_size = input('Enter the value of your portfolio: ')

try: 
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('Please enter an integer')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)
    
    

Enter the value of your portfolio:  10000000


10000000.0


In [26]:
position_size = val/len(final_dataframe.index)
print(f'position_size is {position_size}')
for i in range (0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe
# aapl_shares = math.floor(position_size / 503)
# print(f'aapl_shares is {aapl_shares}')


position_size is 19880.715705765408


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,99.63,55131382035,199
1,AOS,86.06,13147861854,231
2,ABT,104.77,182261428930,189
3,ABBV,161.28,284799155720,123
4,ACN,307.41,206094347708,64
...,...,...,...,...
498,XYL,141.27,34250524844,140
499,YUM,136.38,38409001073,145
500,ZBRA,316,16248531348,62
501,ZBH,119.59,24603059117,166


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [32]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False) 

C:\Users\tuanm\AppData\Local\Temp\ipykernel_46204\4102218004.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  final_dataframe.to_excel(writer, 'Recommended Trades', index = False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [33]:
background_color = '0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [34]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [36]:
writer.close()